In [17]:
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn import metrics
import logging
import os
import sys
from cgi import test
from glob import glob
from pydoc import describe
import numpy as np
import pandas as pd
from joblib import dump, load
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import (
    BayesianRidge,
    ElasticNet,
    LinearRegression,
    SGDRegressor,
)
from sklearn.model_selection import train_test_split
from sklearn.multioutput import RegressorChain
# from sklearnex import patch_sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVR
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Dense, LSTM, Conv1D, Dropout, Bidirectional, Multiply, MaxPooling1D, BatchNormalization
from tensorflow.keras.layers import InputLayer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [18]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [19]:
df_pre = pd.read_csv('C:/Users/vp322/Documents/Nam4/NCKH/DATA/Risk_2 - smooth.csv', sep=",", dtype=str)
df_pre = df_pre[['PM2.5_s','wd_s','ws_s','Rainfall_s','lowleaf_s','temp_s','uvb_s']]
df_pre.head()

,PM2.5_s,wd_s,ws_s,Rainfall_s,lowleaf_s,temp_s,uvb_s
0,24.33,349.33,1.37,0.00,1.92,296.92,0.00
1,27.33,341.67,1.37,0.00,1.92,296.40,0.00
2,31.33,342.00,1.40,0.00,1.92,295.95,0.00
3,33.00,346.33,1.33,0.00,1.92,295.94,4580.61
4,36.67,245.00,1.20,0.00,1.92,296.41,29870.49


In [20]:
values = df_pre.values
# ensure all data is float
values = values.astype('float32')
temp = pd.DataFrame(values)
temp.head()

,0,1,2,3,4,5,6
0,24.330000,349.329987,1.37,0.0,1.92,296.920013,0.000000
1,27.330000,341.670013,1.37,0.0,1.92,296.399994,0.000000
2,31.330000,342.000000,1.40,0.0,1.92,295.950012,0.000000
3,33.000000,346.329987,1.33,0.0,1.92,295.940002,4580.609863
4,36.669998,245.000000,1.20,0.0,1.92,296.410004,29870.490234


In [21]:
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [22]:
#Single timestep no scale
# frame as supervised learning  
n_hours = 18 # check timestep change  timestep to evaluate performance


n_features = 7  # fixed 7 values in data  (PM2.5_s	wd_s	ws_s	Rainfall_s	lowleaf_s	temp_s	uvb_s)
reframed = series_to_supervised(scaled, n_hours, 1)  # apply scaled not values
reframed.head()

,var1(t-18),var2(t-18),var3(t-18),var4(t-18),var5(t-18),var6(t-18),var7(t-18),var1(t-17),var2(t-17),var3(t-17),...,var5(t-1),var6(t-1),var7(t-1),var1(t),var2(t),var3(t),var4(t),var5(t),var6(t),var7(t)
18,0.152534,0.971198,0.183261,0.0,0.466667,0.222088,0.000000,0.172148,0.949861,0.183261,...,0.466667,0.407566,0.0,0.150376,0.405738,0.307359,0.0,0.466667,0.370956,0.0
19,0.172148,0.949861,0.183261,0.0,0.466667,0.190359,0.000000,0.198300,0.950780,0.187590,...,0.466667,0.370956,0.0,0.152534,0.363955,0.259740,0.0,0.466667,0.341673,0.0
20,0.198300,0.950780,0.187590,0.0,0.466667,0.162905,0.000000,0.209219,0.962841,0.177489,...,0.466667,0.341673,0.0,0.176528,0.284123,0.216450,0.0,0.466667,0.311777,0.0
21,0.209219,0.962841,0.177489,0.0,0.466667,0.162294,0.010598,0.233213,0.680585,0.158730,...,0.466667,0.311777,0.0,0.226675,0.173621,0.183261,0.0,0.466667,0.281271,0.0
22,0.233213,0.680585,0.158730,0.0,0.466667,0.190971,0.069108,0.198300,0.399248,0.139971,...,0.466667,0.281271,0.0,0.268061,0.402033,0.177489,0.0,0.466667,0.248932,0.0


In [23]:
# split into train and test sets
values = reframed.values
n_train_hours = 32469 - n_hours -1  # het nam 2021
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]   # test dau nam 2022 den 2023

In [24]:
# split into input and outputs
n_obs = n_hours * n_features
print(n_obs)
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]
print(train_X.shape, len(train_X), train_y.shape)


126
(32450, 126) 32450 (32450,)


In [25]:
# split into train and test sets
values = reframed.values
n_train_hours = 32469 - n_hours - 1  # het nam 2021
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]   # test dau nam 2022 den 2023


In [26]:
# split into input and outputs
n_obs = n_hours * n_features
print(n_obs)
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]
print(train_X.shape, len(train_X), train_y.shape)

126
(32450, 126) 32450 (32450,)


In [27]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(32450, 18, 7) (32450,) (11142, 18, 7) (11142,)


In [28]:
print(train_X.shape[1])
print(train_X.shape[2])

18
7


In [29]:
def LSTMandCNN(input_shape):
    inputs = Input(shape=input_shape)
    
    # Convolution layers with Batch Normalization
    x = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Conv1D(filters=128, kernel_size=3, activation='relu')(x)
    x = BatchNormalization()(x)
    
    # LSTM layers
    bi1 = Bidirectional(LSTM(64, activation='tanh', return_sequences=True))(x)
    bi2 = Bidirectional(LSTM(32, activation='tanh', return_sequences=False))(bi1)
    
    # Dropout to prevent overfitting
    dropout = Dropout(0.3)(bi2)
    
    # Output layer
    dense3 = Dense(1)(dropout)
    
    model = Model(inputs, dense3)
    return model

# Khởi tạo mô hình
model = LSTMandCNN(input_shape=(train_X.shape[1], train_X.shape[2]))
model.summary()

# Compile mô hình với optimizer đã điều chỉnh
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='mse')
'''
# model 2
def LSTMandCNN(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv1D(filters=128, kernel_size=1, activation='relu')(inputs)
    bi1 = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(x)
    max1d= tf.keras.layers.GlobalAveragePooling1D()(bi1)
    #dropout = Dropout(0.2)(max1d)
    #dense1=Dense(128)(dropout)
    #dense2=Dense(64)(dense1)
    dense3=Dense(1)(max1d)
    model=Model([inputs],dense3)
    return model

model = LSTMandCNN(input_shape=(train_X.shape[1], train_X.shape[2]))
model.summary()
model.compile(optimizer = 'adam',loss = 'mse')
'''

'''
Test RMSE: 3.5072
Test MAE: 2.5244
Test R2: 0.9203
'''

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 18, 7)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 16, 64)         │         1,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 14, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 14, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 14, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 64)             │        41,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 166,977 (652.25 KB)

 Trainable params: 166,593 (650.75 KB)

 Non-trainable params: 384 (1.50 KB)

In [ ]:
from keras.callbacks import EarlyStopping

history = model.fit(train_X, train_y, 
                    epochs=67, 
                    batch_size=32, 
                    validation_split=0.2, 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=30, verbose=1,
                               mode='min', restore_best_weights=True)])
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

Epoch 1/67
812/812 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.0468 - val_loss: 0.0042
Epoch 2/67
265/812 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.0132

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, -(n_features-1):]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:, 0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -(n_features-1):]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:, 0]

In [ ]:
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.4f' % rmse)
              
#calculate MAE
mae = mean_absolute_error(inv_y, inv_yhat)
print('Test MAE: %.4f' % mae)
              
# r-squared score of the model
r2 = r2_score(inv_y, inv_yhat)
print('Test R2: %.4f'% r2)

In [ ]:
#Plot the graph between actual vs predicted values
plt.figure(figsize=(10,6))  
plt.plot(inv_yhat[:500], color= 'blue',label = 'Predicted Pollution level')
plt.plot(inv_y[:500] , color = 'red',label = 'Actual Pollution level')
plt.title("Air Pollution Prediction (Multivariate)")
plt.xlabel("Date")
plt.ylabel("Pollution level")
plt.legend()
plt.show()
plt.savefig('graph.png')

In [ ]:
#write files
output = DataFrame([inv_y,inv_yhat]).T
#output.columns = ['actual','predicted']
#output.to_csv("LSTM"+"_"+"PM25_prediction_"+"timestep_"+str(n_hours)+".csv")
output